In [ ]:
# Laura Chapman
# Script for developing statistical tools for Glue
# Computes statistics for subsets as well as entire data using compute_statistic
# Changed from astropy tables to pandas dataframe

# Format data well in a popup using qt
# Color code by subset

In [ ]:
# Basic code that imports glue and loads in and links the data

import sys
from glue.core.data_factories import load_data
from glue.core import DataCollection
from glue.core.link_helpers import LinkSame
from glue.app.qt.application import GlueApplication
from glue.viewers.image.qt import ImageViewer
from glue_vispy_viewers.volume.volume_viewer import VispyVolumeViewer

image_filename='w5.fits'
catalog_filename='w5_psc.vot'

#load 2 datasets from files
catalog = load_data(catalog_filename)
image = load_data(image_filename)

dc = DataCollection([catalog,image])

# link positional information
dc.add_link(LinkSame(catalog.id['RAJ2000'], image.id['Right Ascension']))
dc.add_link(LinkSame(catalog.id['DEJ2000'], image.id['Declination']))

#Create subset based on filament mask
ra_state=(image.id['Right Ascension'] > 44) & (image.id['Right Ascension'] < 46)
subset_group=dc.new_subset_group('RA_Selection',ra_state)
subset_group.style.color = '#0000FF'



In [ ]:
import glue.utils.array as gua
import glue.core.data as gcd
from astropy.table import Table

In [ ]:
# Constructs a pandas DataFrame instead of an astropy table

import pandas as pd
from pandas import DataFrame
import numpy as np

In [ ]:
# Defines a subset and runs statistics using compute_statistic for the subset
# Saves data using a pandas dataframe called my_pandas_data

# Data that subset is pulled from
data = dc[0]

# Define state and subset
state1 = data.id['Jmag'] > 14
subset1 = data.new_subset(state1, label='Jmag > 14')

# Same arrays as with full data
mean_array = []
median_array = []
min_array = []
max_array = []
sum_array = []
name_array = []
tables = []

headings = ('mean', 'median', 'minimum', 'maximum', 'sum')

print(data.label, 'subset1')
print() 
for j in range (0, len(data.components)):
    name = data.components[j].label # Get the name of each component
    name_array.append(name) # add to the name array to build the table
    mean_array.append(data.compute_statistic('mean', subset1.components[j], subset_state=subset1.subset_state))
    median_array.append(data.compute_statistic('median', subset1.components[j], subset_state=subset1.subset_state))       
    min_array.append(data.compute_statistic('minimum', subset1.components[j], subset_state=subset1.subset_state))       
    max_array.append(data.compute_statistic('maximum', subset1.components[j], subset_state=subset1.subset_state))      
    sum_array.append(data.compute_statistic('sum', subset1.components[j], subset_state=subset1.subset_state))        
   
column_data = np.asarray([mean_array, median_array, min_array, max_array, sum_array]).transpose()

my_pandas_data = pd.DataFrame(column_data, index=name_array, columns=headings)

In [ ]:
from PyQt5.QtWidgets import QApplication, QComboBox, QWidget, QPushButton, QHBoxLayout, QTableView,QGroupBox, QDialog, QVBoxLayout, QLabel,QGridLayout
from PyQt5.QtGui import QIcon
from PyQt5.QtCore import pyqtSlot
from PyQt5 import QtCore
import sys

class AbsolutePositioningExample(QWidget):
    ''' An example of PySide absolute positioning; the main window
        inherits from QWidget, a convenient widget for an empty window. '''
    def __init__(self,dc):
        # Initialize the object as a QWidget
        QWidget.__init__(self)
        
        self.dc=dc
 
        self.setMinimumSize(800, 600)
        self.setMaximumSize(800, 600)

        self.setWindowTitle('Statistics')
        
        self.choose_data = QComboBox(self)
        self.choose_data.addItems(["Choose a dataset"])
        self.choose_data.addItems(dc.labels)
        self.choose_data.model().item(0).setEnabled(False)
        self.choose_data.move(300, 5)
        self.choose_data.currentIndexChanged.connect(self.selectionchange)
        
        self.table = QTableView(self)
        self.table.setGeometry(50, 250, 700, 300) 


        self.horizontalGroupBox = QGroupBox(self)
        self.horizontalGroupBox.setGeometry(50, 50, 700, 200) 

        
    def clicked(self):
        button=self.sender()
        if isinstance(button, QPushButton):
            if button.isChecked():
                print("button pressed")
            else:
                print('button released')

                
    def selectionchange(self,data_index):
        
        try:
            self.component_layout
            while self.component_layout.count():
                item = self.component_layout.takeAt(0)
                widget = item.widget()
                widget.deleteLater()
        except:
            self.component_layout = QGridLayout()
            
        ncols=3
        nrows=int(np.round((len(self.dc[data_index-1].components)/3.)+0.5))

        i=0
        for col in range(0,nrows):
            for row in range(0,ncols):
                btnlabel=str(self.dc[data_index-1].components[i])
                btn=QPushButton(btnlabel)
                btn.setCheckable(True)
                btn.toggle()
                btn.clicked.connect(self.clicked)
                self.component_layout.addWidget(btn,col,row)
                btn.setChecked(False)
                i=i+1
                
                if i==len(dc[data_index-1].components):
                    break    
                
        self.horizontalGroupBox.setLayout(self.component_layout)

        
app = QApplication.instance()
if app is None:
    app = QApplication(sys.argv)
else:
    print('QApplication instance already exists: %s' % str(app))
ex = AbsolutePositioningExample(dc)
ex.show()
sys.exit(app.exec_())